In [11]:
import requests
from bs4 import BeautifulSoup

#Scrape the source code from the specified link:
#In this example I'm using my own medium wall
data = requests.get("https://fabiom91.medium.com")
soup = BeautifulSoup(data.content, 'html.parser')

In [12]:
import re

keywords= ['privacy','data','analytics','data science','cybersecurity']
urls = []
for key in keywords:
    html = str(soup.find_all('a', attrs = {'href':re.compile(r'^.*\b%s\b.*$' % key)}))
    href_start = [s.start() for s in re.finditer('href="',html)]
    for start in href_start:
        url = html[start+6:]
        url = url[:url.find('"')]
        if not any(c in '#?^%*()=' for c in url):
            if 'http' in url:
                urls.append(url)
            else:
                urls.append("https://fabiom91.medium.com" + url)
            
urls = list(set(urls))

In [13]:
from tqdm import tqdm
previews = []
for article in tqdm(urls):
    data = requests.get(article)
    soup = BeautifulSoup(data.content, 'html5lib')
    
    title = soup.find('h1').contents[0]
    subtitle = soup.find('p').contents[0]
    subtitle = BeautifulSoup(str(subtitle), 'lxml').text
    for x in soup.find_all('img'):
        try:
            image = str(x.attrs['srcset']).split(" ")[0]
            break
        except:
            pass
        
    article_preview = {
        'title': str(title),
        'subtitle': str(subtitle),
        'image': str(image)
    }
    previews.append(article_preview)

0it [00:00, ?it/s]


In [14]:
print(previews)

[]


In [15]:
template = open('index.html')
soup = BeautifulSoup(template.read(), "html.parser")

In [16]:
article_template = soup.find('div', attrs={'class':'columns'})
html_start = str(soup)[:str(soup).find(str(article_template))]
html_end = str(soup)[str(soup).find(str(article_template))+len(str(article_template)):]
html_start = html_start.replace('\n','')
html_end = html_end.replace('\n','')

In [17]:
newsletter_content = ""
for i,article in enumerate(previews):
    
    try:
        img = article_template.img
        img['src'] = article['image']
        article_template.img.replace_with(img)
    except:
        pass
    
    title = article_template.h1
    title.string = article['title'][:300]
    
    subtitle = article_template.p
    subtitle.string = article['subtitle'][:300] + "..."
    
    link = article_template.a
    link['href'] = urls[i]
    link.string = urls[i]
    article_template.a.replace_with(link)
    
    
    newsletter_content += str(article_template).replace('\n','')

email_content = html_start + newsletter_content + html_end

In [18]:
print(BeautifulSoup(email_content).prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://cdn.jsdelivr.net/npm/bulma@0.9.1/css/bulma.min.css" rel="stylesheet"/>
 </head>
 <body>
  <section class="section">
  </section>
 </body>
</html>


In [19]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [21]:
sender_email = "go.reply.newsletter@gmail.com"
receiver_email = "sanskargupta15@gmail.com"
# sender email password
# f=open("pw.txt","r")
# lines=f.readlines()
# password=lines[0]
# f.close()
password = "cyqmprvmqutyoxww"

message = MIMEMultipart("alternative")
message["Subject"] = "My awesome newsletter"
message["From"] = sender_email
message["To"] = receiver_email

# Create the plain-text and HTML version of your message
text = "Hi, I've found some article that you might find interesting: %s" % previews
html = email_content

# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.ehlo()
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )

SMTPAuthenticationError: (534, b'5.7.9 Please log in with your web browser and then try again. Learn more at\n5.7.9  https://support.google.com/mail/?p=WebLoginRequired sc42-20020a1709078a2a00b00722e50dab2csm2105318ejc.109 - gsmtp')